# Use CKAN API to add datasets and organisations

Documentation: https://docs.ckan.org/en/latest/api/index.html

### Setup

In [121]:
import os
import pprint
import json

import requests
from dotenv import dotenv_values

config = {
    # **dotenv_values(".env"),  # load shared development variables
    **dotenv_values(".env_local"),  # load sensitive variables
    # **os.environ,  # override loaded values with environment variables
}

In [184]:
# headers = {'Authorization': config['API_TOKEN']}

headers = {'Authorization': config['API_TOKEN'], 
           'Content-Type': 'application/json'}

In [123]:
# base_url = 'http://data.climatesubak.org/api/3/action/'
base_url = 'http://nuc:5000/api/3/action/'

### Basic queries

In [53]:
def get_all_orgs():
    url = base_url + 'organization_list'
    r = requests.get(url, headers=headers)
    return r.json()

In [54]:
def get_licenses():
    url = base_url + 'license_list'
    r = requests.get(url, headers=headers)
    return r.json()

In [96]:
def get_package_list_with_resources():
    url = base_url + 'current_package_list_with_resources'
    r = requests.get(url, headers=headers)
    return r.json()

In [102]:
requests.get(base_url + 'package_list', headers=headers).json()

{'help': 'http://data.climatesubak.org/api/3/action/help_show?name=package_list',
 'success': True,
 'result': ['global-power-plant-database',
  'global_electricity',
  'ids',
  'renewable-energy-planning-database']}

In [103]:
requests.get(base_url + 'package_list_with_resources', headers=headers).json()

'Bad request - Action name not known: package_list_with_resources'

## Get a single package

In [120]:
with open('power_plant_ids.json', 'w') as outfile:
    json.dump(requests.get(base_url + 'package_show?id=ids', headers=headers).json()['result'], outfile)
    
f = open('power_plant_ids.json')
data = json.load(f)

# data

## Create Organisations

In [195]:
with open('organisations.json') as f:
    orgs = json.loads(f.read())
    
# orgs['organisations']

In [224]:
url = base_url + 'organization_create'

for org in orgs['organisations']:
    r = requests.post(url, data=json.dumps(org), headers=headers)
    print(r.json())

In [199]:
get_all_orgs()

{'help': 'http://nuc:5000/api/3/action/help_show?name=organization_list',
 'success': True,
 'result': ['test_org',
  'climatepolicyradar',
  'subak',
  'dc_admin',
  'ember',
  'newautomotive',
  'openclimatefix',
  'test',
  'transitionzero']}

In [16]:
# get_licenses()

# All datasets (packages) to json
Save to `all_packages.json` file

In [49]:
with open('all_packages.json', 'w') as outfile:
    json.dump(get_package_list_with_resources()['result'], outfile)

# Create all datasets from package file

First edit `all_packages.json` to delete all the specific `id` values.  
Then ensure there is an `owner_org` for each record. Use the org name or `dc_admin` if org is not present.

In [223]:
url = base_url + 'package_create'
with open('all_packages.json') as f:
    data = json.loads(f.read())
    for package in data:
        r = requests.post(url, data=json.dumps(package), headers=headers)
        print(r.json())

{'help': 'http://nuc:5000/api/3/action/help_show?name=package_create', 'error': {'name': ['That URL is already in use.'], '__type': 'Validation Error'}, 'success': False}
{'help': 'http://nuc:5000/api/3/action/help_show?name=package_create', 'error': {'name': ['That URL is already in use.'], '__type': 'Validation Error'}, 'success': False}
{'help': 'http://nuc:5000/api/3/action/help_show?name=package_create', 'error': {'name': ['That URL is already in use.'], '__type': 'Validation Error'}, 'success': False}
{'help': 'http://nuc:5000/api/3/action/help_show?name=package_create', 'error': {'name': ['That URL is already in use.'], '__type': 'Validation Error'}, 'success': False}


## Delete all datasets (leave orgs)

This is probably easiest to do with `ckanapi`

In [207]:
# url = 'http://data.climatesubak.org'
url = 'http://nuc:5000/'
api_key = config['API_TOKEN']

In [221]:
!ckanapi action package_list -j -r $url -a $api_key | ckanapi delete datasets -r $url -a $api_key

1 [2] --- None "global-power-plant-database"
2 [3] 0.09s None "global_electricity"
3 [4] 0.12s None "ids"
4 [None] 0.10s None "renewable-energy-planning-database"


# Dump all orgs to json

# Dump all datasets to json

## Create relationships between packages
`ckan.logic.action.create.package_relationship_create()`